<a href="https://colab.research.google.com/github/cisimon7/ContactDetector/blob/master/Notebooks/Contact_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>Contact Detection Algorithm for a Robot</b></h1>
<h3><i>...</i></h3>

###<b>Imports</b>

In [ ]:
from IPython.display import clear_output 
!pip install casadi
clear_output()

In [ ]:
import numpy as np
import casadi as csd
from scipy import signal
from functools import reduce
from casadi import MX, SX, DM, Function
np.set_printoptions(linewidth=200, precision=4, suppress=True)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

###<b>Utility Functions</b>

#####<i>3x3 Zero Matrix Definition</i>

In [ ]:
zero_mat = SX.zeros((3,3))
eye_mat  = SX.ones((3,3))

#####<i>Rotation Matrices Functions</i>

In [ ]:
def Rx(q):
  T = csd.vertcat(
          csd.horzcat(1,         0,            0, 0),
          csd.horzcat(0, csd.cos(q), -csd.sin(q), 0),
          csd.horzcat(0, csd.sin(q),  csd.cos(q), 0),
          csd.horzcat(0,         0,            0, 1)
      )
  return T

def Ry(q):
  T = csd.vertcat(
          csd.horzcat( csd.cos(q), 0, csd.sin(q), 0),
          csd.horzcat(         0,  1,          0, 0),
          csd.horzcat(-csd.sin(q), 0, csd.cos(q), 0),
          csd.horzcat(         0,  0,          0, 1)
      )
  return T

def Rz(q):
  T = csd.vertcat(
          csd.horzcat(csd.cos(q), -csd.sin(q), 0, 0),
          csd.horzcat(csd.sin(q),  csd.cos(q), 0, 0),
          csd.horzcat(        0,           0,  1, 0),
          csd.horzcat(        0,           0,  0, 1)
      )
  return T

def R_xyz(qx, qy, qz):
  return (Rx(qx) @ Ry(qy) @ Rz(qz))

#####<i>Rotation Derivative Matrices</i>

In [ ]:
def dRx(q):
  T = csd.vertcat(
          csd.horzcat(0,          0,            0, 0),
          csd.horzcat(0, -csd.sin(q), -csd.cos(q), 0),
          csd.horzcat(0,  csd.cos(q), -csd.sin(q), 0),
          csd.horzcat(0,          0,            0, 0)
       )
  return T

def dRy(q):
  T = csd.vertcat(
          csd.horzcat(-csd.sin(q), 0,  csd.cos(q), 0),
          csd.horzcat(         0,  0,           0, 0),
          csd.horzcat(-csd.cos(q), 0, -csd.sin(q), 0),
          csd.horzcat(         0,  0,           0, 0)
      )
  return T

def dRz(q):
  T = csd.vertcat(
          csd.horzcat(-csd.sin(q), -csd.cos(q), 0, 0),
          csd.horzcat( csd.cos(q), -csd.sin(q), 0, 0),
          csd.horzcat(         0,          0,   0, 0),
          csd.horzcat(         0,          0,   0, 0)
      )
  return T

def R_dxyz(dqx, qy, qz):
  return (dRx(dqx) @ Ry(qy) @ Rz(qz))

def R_xdyz(qx, dqy, qz):
  return (Rx(qx) @ dRy(dqy) @ Rz(qz))

def R_xydz(qx, qy, dqz):
  return (Rx(qx) @ Ry(qy) @ dRz(dqz))

#####<i>Translation Matrices</i>

In [ ]:
def Tx(x):
  T = csd.vertcat(
          csd.horzcat(1, 0, 0, x),
          csd.horzcat(0, 1, 0, 0),
          csd.horzcat(0, 0, 1, 0),
          csd.horzcat(0, 0, 0, 1)
      )
  return T

def Ty(y):
  T = csd.vertcat(
          csd.horzcat(1, 0, 0, 0),
          csd.horzcat(0, 1, 0, y),
          csd.horzcat(0, 0, 1, 0),
          csd.horzcat(0, 0, 0, 1)
      )
  return T

def Tz(z):
  T = csd.vertcat(
          csd.horzcat(1, 0, 0, 0),
          csd.horzcat(0, 1, 0, 0),
          csd.horzcat(0, 0, 1, z),
          csd.horzcat(0, 0, 0, 1)
      )
  return T

def T_xyz(x, y, z):
  return (Tx(x) @ Ty(y) @ Tz(z))

#####<i>Translation Derivative Matrices</i>

In [ ]:
def dTx(x):
  T = csd.vertcat(
          csd.horzcat(0, 0, 0, 1),
          csd.horzcat(0, 0, 0, 0),
          csd.horzcat(0, 0, 0, 0),
          csd.horzcat(0, 0, 0, 0)
      )
  return T

def dTy(y):
  T = csd.vertcat(
          csd.horzcat(0, 0, 0, 0),
          csd.horzcat(0, 0, 0, 1),
          csd.horzcat(0, 0, 0, 0),
          csd.horzcat(0, 0, 0, 0)
      )
  return T

def dTz(z):
  T = csd.vertcat(
          csd.horzcat(0, 0, 0, 0),
          csd.horzcat(0, 0, 0, 0),
          csd.horzcat(0, 0, 0, 1),
          csd.horzcat(0, 0, 0, 0)
      )
  return T

def T_dxyz(dpx, py, pz):
  return (dTx(dpx) @ Ty(py) @ Tz(pz))

def T_xdyz(px, dpy, pz):
  return (Tx(px) @ dTy(dpy) @ Tz(pz))

def T_xydz(px, py, dpz):
  return (Tx(px) @ Ty(py) @ dTz(dpz))

#####<i>Calculus Functions</i>

In [ ]:
def Mat_derivative(mat, var):
  """Accepts a matrix, and finds its element-wise derivative with respect to var"""
  assert var.shape == (1,1), "DIFFERENTIATING NOT WITH RESPECT TO A SINGLE VARIABLE, TRY Mat_gradient FUNCTION"
  n, m = mat.shape
  return csd.vertcat(*[ csd.horzcat(*[csd.gradient(mat[i,j], var) for j in range(m)]) for i in range(n) ])

In [ ]:
def Mat_gradient(mat, vars, dvars):
  """Accepts a matrix and finds the element-wise gradient with respect to the vars variables, then multiplies by dvars"""
  n, m = mat.shape
  return csd.vertcat(*[ csd.horzcat(*[csd.dot(csd.gradient(mat[i,j], vars),dvars) for j in range(m)]) for i in range(n) ])

###<b>Jacobian Function</b>


In [ ]:
def Jacobian(FKinematics, jParams):
  R = FKinematics[0:3,0:3]
  J = []

  for var in jParams:
    Td  = csd.simplify(Mat_derivative(FKinematics, var))

    T   = Td[0:3, -1]          #Jv : Linear Velocity Jacobian
    Rd  = Td[0:3,0:3]
    Rj  = Rd @ R.T             #Jw : Angular Velocity Jacobian in SkewMatrix form

    Jvar = csd.vertcat(T, csd.vertcat(Rj[2,1], Rj[0,2], Rj[1,0]))   
    J.append(Jvar)
  
  return csd.horzcat(*J)

###<b>Dynamics Function</b>


In [ ]:
def InertiaMoment(mass, radius, length):
  """Returns the Moment of Inertia Matrix of a cylindrical link, give its dimensions"""
  m, r, h = mass, radius, length
  return np.array([
      [ m/12*(3*r**2 + h**2),                    0,          0 ],
      [                    0, m/12*(3*r**2 + h**2),          0 ],
      [                    0,                    0, 0.5*m*r**2 ]
  ])

#####<i>Inertial Matrix</i>


In [ ]:
def LinkInertia(FKinematics, mass, mInertia, jParams):
  """
  This returns the inertia matrix for a single rigid link, given the Forward 
  Kinematics of its centre of mass, the mass of the link, its moment of Inertia 
  matrix and parameters describing it and other links connected to it
  """
  J = Jacobian(FKinematics, jParams)
  Jv, Jw = J[0:3,:], J[3:6,:]
  R = FKinematics[0:3,0:3]
  I = mInertia
  m = mass

  D = (m * Jv.T @ Jv) + (Jw.T @ R @ I @ R.T @ Jw)
  return D


def InertiaBranch(fkList, massList, mInertList, jParams):
  """
  Receives a list of FK of all the centres, list of all COM, list of all Moment of
  Inertia Matrix, all parameters, then returns the Inertia
  """
  hList = [ LinkInertia(fkList[i], massList[i], mInertList[i], jParams) for i in range(len(massList)) ]
  H = reduce( lambda h1,h2:h1+h2, hList)
  return csd.simplify(H)

#####<i>Coriolis Matrix</i>


In [ ]:
def CoriolisBranch(Hmat, qdots, q):
  x, y = Hmat.shape
  n = len(q)
  C = SX.zeros(x,y)

  for k in range(n):
      for j in range(n):
          C_kj = 0
          for i in range(n):
              c_ijk = 0.5 * (csd.gradient(Hmat[k, j], q[i]) + csd.gradient(Hmat[k, i], q[j]) - csd.gradient(Hmat[i, j], q[k]))
              C_kj = C_kj + c_ijk*qdots[i]
          C[k, j] = C_kj

  return csd.vertcat(*[ csd.horzcat(*[C[i,j] for j in range(y)]) for i in range(x) ])

#####<i>Gravity Vector</i>


In [ ]:
def PotentialBranch(fkcentres, q, masses, gravity=[0,0,9.81]):
  n=len(fkcentres)
  pos = [ cent[0:3,-1] for cent in fkcentres ]
  Potentials = [ masses[i]*csd.dot(pos[i], gravity) for i in range(n) ]
  PE = reduce(lambda p1,p2: p1+p2, Potentials)
  G = csd.gradient(PE,csd.vertcat(*q))
  
  return G

###<b>Friction Models</b>


#####<i>Stribeck Friction Model</i>


In [ ]:
class StribeckFrictionModel():
  def __init__(self, coeffs):
    self.coeffs= coeffs
  
  def instance(self, joint_angles, joint_velocities):
    q, dq = csd.vertcat(*joint_angles), csd.vertcat(*joint_velocities)
    a, b, c, d, e, f, g, h = self.coeffs
    model = a*dq + b*csd.vertcat(1,1,1) + c*csd.exp(-d*csd.power(dq,2)) + e*csd.sin(q) + f*csd.cos(q) + g*csd.sin(2*q) + h*csd.cos(2*q)
    
    return model

#####<i>Coulomb-Viscous Friction Model</i>


In [ ]:
class CoulombViscousFrictionModel():
  def __init__(self, static_friction, coulomb_coeff, viscous_coeff):
    self.s_friction = static_friction
    self.c_coeff = coulomb_coeff
    self.v_coeff = viscous_coeff
  
  def instance(self, velocity, external_force, angle):
    v, e_force = velocity, external_force
    return csd.conditional(
        all([v==0, csd.fabs(e_force)<self.s_friction]),
        e_force,
        csd.conditional(
            all([v==0, csd.fabs(e_force)>=self.s_friction]),
            self.s_friction,
            self.c_coeff*e_force*csd.sin(angle)*csd.sign(v) + self.v_coeff*v
        )
    )

###<b>Torque Observer</b>


In [ ]:
class ExternalTorqueEstimator():
  """ETOGainMatrix represents External Torque Observer Gain Matrix"""
  def __init__(self, time_step, vars, d_vars, ETOGainMatrix, matrix_inertia, coriolis_matrix, gravity_vector):
    self.step = time_step
    self.vars = vars
    self.d_vars = d_vars
    self.ETOGainMatrix = ETOGainMatrix
    self.m_inertia = matrix_inertia
    self.c_matrix = coriolis_matrix
    self.g_vector = gravity_vector
  
  def instance(self):
    dof = len(self.vars)
    Kr, K, h = self.ETOGainMatrix, SX.sym('control_gain'), self.step
    M, C, G = self.m_inertia, self.c_matrix, self.g_vector
    q, dq = self.vars, self.d_vars
    q_prev, dq_prev = [SX.sym(f'q_prev-{i}') for i in range(dof)], [SX.sym(f'dq_prev-{i}') for i in range(dof)]
    τ, τ_prev = [SX.sym(f'joint_torques-{i}') for i in range(dof)], [SX.sym(f'joint_torques_prev-{i}') for i in range(dof)]
    
    P = M@csd.vertcat(*self.d_vars)
    dpHat = csd.Function('dpHat',[*self.vars, *self.d_vars],[K*(csd.vertcat(*τ) + csd.transpose(C)@csd.vertcat(*self.d_vars) - G) - csd.vertcat(*τ_prev)])
    collision_judgment_index = Kr * ((h/2)*(dpHat(*q, *dq)+dpHat(*q_prev, *dq_prev)) - P)

    return csd.Function(
        'collision_judgment_index',
        [SX.sym('time'), K, *q, *dq, *q_prev, *dq_prev, *τ, *τ_prev], 
        [collision_judgment_index]
    )

In [ ]:
def isCollision(external_torque, friction_torque, threshold=0.01):
  return csd.if_else(
      csd.norm_2(external_torque - friction_torque) >= threshold,
      True,
      False
  )

###<b>Example on a three Link Manipulator</b>


#####<i>Variables and Structure of Manipulator</i>


In [ ]:
m1, m2, m3 =   1,   1,   1
l1, l2, l3 = 0.5, 0.5, 0.5
q1,   q2,  q3 =  SX.sym('θ1'),  SX.sym('θ2'),  SX.sym('θ3')
dq1, dq2, dq3 = SX.sym('dθ1'), SX.sym('dθ2'), SX.sym('dθ3')

joint_angles, joint_velocities = [q1, q2, q3], [dq1, dq2, dq3]

In [ ]:
inertias = [ InertiaMoment(m1, 0.15, l1), InertiaMoment(m2, 0.15, l2), InertiaMoment(m3, 0.15, l3) ]

#####<i>Manipulator Kinematics</i>


In [ ]:
FKinematics = Rx(q1) @ Tx(l1) @ Rx(q2) @ Tx(l2) @ Rx(q3) @ Tx(l3)

In [ ]:
FKCentres = [ 
    Rx(q1) @ Tx(0.5*l1),
    Rx(q1) @ Tx(l1) @ Rx(q2) @ Tx(0.5*l2),
    Rx(q1) @ Tx(l1) @ Rx(q2) @ Tx(l2) @ Rx(q3) @ Tx(0.5*l3)
 ]

#####<i>Manipulator Jacobian</i>

In [ ]:
jacobian = Jacobian(FKinematics, [q1, q2, q3])

In [ ]:
csd.Function('Jacobian',[*joint_angles],[jacobian])(np.pi/3,np.pi/3,np.pi/3)

DM(
[[0, 0, 0], 
 [0, 0, 0], 
 [0, 0, 0], 
 [1, 1, 1], 
 [0, 0, 0], 
 [0, 0, 0]])

#####<i>Manipulator Inertia Matrix</i>

In [ ]:
inertia_matrix = InertiaBranch(FKCentres, [m1,m2,m3], inertias, [q1, q2, q3])

In [ ]:
csd.Function('InertiaBranch',[*joint_angles],[inertia_matrix])(np.pi/3,np.pi/3,np.pi/3)

DM(
[[0.079375, 0.0529167, 0.0264583], 
 [0.0529167, 0.0529167, 0.0264583], 
 [0.0264583, 0.0264583, 0.0264583]])

#####<i>Manipulator Coriolis Matrix</i>

In [ ]:
coriolis_matrix = CoriolisBranch(inertia_matrix, [dq1,dq2,dq3],[q1,q2,q3])

In [ ]:
csd.Function('CoriolisBranch',[*joint_angles,*joint_velocities],[coriolis_matrix])(np.pi/3,np.pi/3,np.pi/3,0.1,0.1,0.1)

DM(
[[-1.38778e-18, -2.50373e-18, -1.46289e-18], 
 [6.93889e-19, -6.93889e-19, -1.31266e-18], 
 [1.56125e-18, 1.04083e-18, 5.20417e-19]])

#####<i>Manipulator Gravity Vector</i>

In [ ]:
gravity_vector = PotentialBranch(FKCentres, [q1,q2,q3], [m1,m2,m3])

In [ ]:
csd.Function('PotentialBranch',[q1, q2, q3],[gravity_vector])(np.pi/3,np.pi/3,np.pi/3)

DM([0, 0, 0])

#####<i>Manipulator Friction Model</i>

In [ ]:
friction_model = StribeckFrictionModel([1,1,1,1,1,1,1,1])
ThreeLinkFrictionModel = csd.Function(
    'StribeckFrictionModel',
    [*joint_angles, *joint_velocities],
    [friction_model.instance(joint_angles, joint_velocities)]
)

In [ ]:
ThreeLinkFrictionModel(*[ np.pi/3 for _ in range(6) ])

DM([4.11325, 4.11325, 4.11325])

#####<i>Manipulator Torque Estimator Function</i>

In [ ]:
ThreeLinkExternalTorqueEstimator = ExternalTorqueEstimator(
  time_step=0.1,
  vars=joint_angles,
  d_vars=joint_velocities,
  ETOGainMatrix=10**3, 
  matrix_inertia=inertia_matrix, 
  coriolis_matrix=coriolis_matrix, 
  gravity_vector=gravity_vector
)

In [ ]:
(ThreeLinkExternalTorqueEstimator.instance())(*[ 0 for _ in range(20)])

DM([0, 0, 0])

In [ ]:
def ThreeLinkContactDetector(time, control_gain, joint_angles, joint_velocity, joint_angles_prev, joint_velocity_prev, joint_torque, joint_torque_prev):
  estimator = ThreeLinkExternalTorqueEstimator.instance()
  external_torque = estimator(
      time, 
      control_gain,
      *joint_angles, 
      *joint_velocity, 
      *joint_angles_prev, 
      *joint_velocity_prev, 
      *joint_torque, 
      *joint_torque_prev
  )
  friction_torque = ThreeLinkFrictionModel(*joint_angles, *joint_velocity)
  return isCollision(external_torque, friction_torque, threshold=0.01)

#####<i>Joint Torque Model</i>